# NLP to SQL Project

In this project, we'll develop a start-up that can take a non-technical manager's Natural Language question, such as "What counties had the top sales?" and convert that into a SQL query. We will then use that SQL to query the data (which in this example will come from a .csv file we read in with Pandas and set-up a temporary DB in RAM) and then report back the results!

## Imports

In [1]:
import os
import pandas as pd
import openai


## Data


Our data will be some example Sales Data from:  https://www.kaggle.com/datasets/kyanyoga/sample-sales-data

We've already downloaded the dataset for you as "sales_data_sample.csv".

Let's read it in:

In [2]:
df = pd.read_csv("sales_data_sample.csv")

In [3]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Querying Data

We can query via Pandas Syntax:

In [4]:
# Example Pandas Query for Sum of Sales by Quarter 
df.groupby("QTR_ID").sum()['SALES']

/var/folders/rd/v1tq2dh563z8fv2z2v1xs6yw0000gn/T/ipykernel_37142/1598294123.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("QTR_ID").sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64


Or we can query via SQL Syntax, let's set-up a temporary in memory (RAM) database, basically export this CSV-->Pandas DF--> SQL DB

## SQL Database Set-up

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [6]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

Here we push our entire DataFrame to a table called Sales:

In [7]:
data = df.to_sql(name='Sales',con=temp_db)

2023-03-19 18:21:44,876 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-19 18:21:44,877 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-19 18:21:44,878 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-19 18:21:44,878 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-19 18:21:44,878 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-19 18:21:44,879 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-19 18:21:44,881 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-19 18:21:44,882 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	

---

Notice the output above reports some of the SQL commands to create the table as well as the data that went into it.

### Connecting to SQL Database:

Using SQL Alchemy we can establish a connection to this temporary database and query it for the results:

In [8]:
with temp_db.connect() as conn:
    result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))

2023-03-19 18:21:52,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-19 18:21:52,495 INFO sqlalchemy.engine.Engine Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1
2023-03-19 18:21:52,495 INFO sqlalchemy.engine.Engine [generated in 0.00255s] ()
2023-03-19 18:21:52,497 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
result.all()

[(10407, 14082.8)]

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [15]:
# os.environ["OPENAI_API_KEY"] = "YOUR KEY GOES HERE, THEN DELETE THIS LINE OF CODE TO PREVENT ANYONE FROM SEEING YOUR KEY"
# search online in creating a api key for openai and notice that calling api can lead to some cost, and it varies on which model you are calling and max_tokens

In [10]:
os.environ['OPENAI_API_KEY']=''

In [11]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Inform GPT about the SQL Table Structure

We need to tell GPT what the table structure looks like before it can understand the schema enough to create a SQL query. Let's create a function to generate the first part of the example text below (which we can then attach a user natural language query to!)


**Below is an example input to GPT, we tell it the table structure and the NLP question, then at the end we tell it to "SELECT"...**

**Thus GPT must finish with the rest of the most reasonable SQL query.**

In [17]:
### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
# SELECT


In [12]:
def create_table_definition_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables
    """

    prompt = '''### sqlite SQL table, with its properties:
#
# Sales({})
#
'''.format(",".join(str(x) for x in df.columns))
    
    return prompt

In [13]:
print(create_table_definition_prompt(df))

### sqlite SQL table, with its properties:
#
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#



### Get Natural Language Request:

Now let's create a function that grabs the natural language information request:

In [14]:
def prompt_input():
    nlp_text = input("Enter information you want to obtain: ")
    return nlp_text

In [15]:
nlp_text = prompt_input()

Enter information you want to obtain: 


Now let's combine the results in one function:

In [16]:
nlp_text

''

In [17]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition_prompt(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string
    

In [18]:
combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n#\n# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n#\n### A query to answer: \nSELECT'

Notice how GPT works, it will complete the text above, thus we start to notify it to begin a SQL query by writing "\nSELECT"

Now let's get the response:

### OpenAI API Call

Let's use the Code DaVinci model since we know it understand SQL (in the future generalized models may work even better and not need the specify "Code")!

In [19]:
response = openai.Completion.create(
  model="code-davinci-002",
  prompt=combine_prompts(df, nlp_text),
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)


In [20]:
response

<OpenAIObject text_completion id=cmpl-6vweQmuPWl6d1hZGYruGyG95JqxJC at 0x117b86e70> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " CUSTOMERNAME, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY CUSTOMERNAME\nORDER BY TOTAL_SALES DESC\nLIMIT 10"
    }
  ],
  "created": 1679268170,
  "id": "cmpl-6vweQmuPWl6d1hZGYruGyG95JqxJC",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 46,
    "prompt_tokens": 148,
    "total_tokens": 194
  }
}

Now let's build a function to parse the section of the response we want: 

In [21]:
def handle_response(response):
    query = response["choices"][0]["text"]
    if query.startswith(" "):
        query = "Select"+ query
    return query

In [22]:
handle_response(response)

'Select CUSTOMERNAME, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY CUSTOMERNAME\nORDER BY TOTAL_SALES DESC\nLIMIT 10'

Perfect! Now we just pass that into our Database:

In [23]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-19 18:23:03,224 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-19 18:23:03,226 INFO sqlalchemy.engine.Engine Select CUSTOMERNAME, SUM(SALES) AS TOTAL_SALES
FROM Sales
GROUP BY CUSTOMERNAME
ORDER BY TOTAL_SALES DESC
LIMIT 10
2023-03-19 18:23:03,227 INFO sqlalchemy.engine.Engine [generated in 0.00285s] ()
2023-03-19 18:23:03,231 INFO sqlalchemy.engine.Engine ROLLBACK


In [24]:
result.all()

[('Euro Shopping Channel', 912294.1100000002),
 ('Mini Gifts Distributors Ltd.', 654858.06),
 ('Australian Collectors, Co.', 200995.40999999997),
 ('Muscle Machine Inc', 197736.93999999997),
 ('La Rochelle Gifts', 180124.9),
 ('Dragon Souveniers, Ltd.', 172989.68000000008),
 ('Land of Toys Inc.', 164069.44000000003),
 ('The Sharp Gifts Warehouse', 160010.26999999996),
 ('AV Stores, Co.', 157807.80999999997),
 ("Anna's Decorations, Ltd", 153996.13000000003)]

Perfect! You can explore the .py files to view a system that brings it all together!